In [ ]:
import os
import glob

import plotly.graph_objects as go
import polars as pl
import ipywidgets as widgets
from IPython.display import display

DATA_DIR = "./results"

parquet_files = sorted([os.path.basename(p) for p in glob.glob(os.path.join(DATA_DIR, "*.parquet"))])


def add_fig(fig, data, name):
    fig.add_trace(go.Scatter(x=data["generation"], y=data["best_mean"], mode="lines", name=name))
    fig.add_traces(
        go.Scatter(
            x=data["generation"].to_list() + data["generation"].to_list()[::-1],
            y=data["best_upper"].to_list() + data["best_lower"].to_list()[::-1],
            fill="toself",
            name="std one dev",
            line={"width": 0},
            opacity=0.25,
            hoverinfo="skip",
            showlegend=False,
        )
    )


def plot_two_files(baseline_file, relaxed_file):
    fig = go.Figure()

    for f_name, label in [(baseline_file, "Data 1"), (relaxed_file, "Data 2")]:
        data = pl.read_parquet(os.path.join(DATA_DIR, f_name))
        add_fig(fig, data, label)

    fig.update_layout(
        title="Best mean f_1 per generation",
        xaxis_title="Generation",
        yaxis_title="Cumulative best mean f_1",
    )
    fig.show()


baseline_dd = widgets.Dropdown(
    options=parquet_files,
    description="Data 1:",
    value=parquet_files[0] if parquet_files else None,
    layout=widgets.Layout(width="70%"),
)

relaxed_dd = widgets.Dropdown(
    options=parquet_files,
    description="Data 2:",
    value=parquet_files[1] if len(parquet_files) > 1 else None,
    layout=widgets.Layout(width="70%"),
)

button = widgets.Button(description="Plot", button_style="primary")

# Output area for the plot
out = widgets.Output()


def on_click_plot(_):
    with out:
        out.clear_output(wait=True)
        if baseline_dd.value is None or relaxed_dd.value is None:
            print("Select both files first.")
            return
        plot_two_files(baseline_dd.value, relaxed_dd.value)


button.on_click(on_click_plot)

display(widgets.VBox([baseline_dd, relaxed_dd, button, out]))